In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# RL Agent Training Demo\n",
    "This notebook demonstrates training a LinUCB contextual bandit agent on a small subset of data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from core.features import FeaturePipeline\n",
    "from core.models.rl_agent import LinUCBAgent, RLTrainer\n",
    "\n",
    "%matplotlib inline\n",
    "plt.style.use('ggplot')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 1: Load Feature Store"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "pipeline = FeaturePipeline()\n",
    "feature_dict = pipeline.load_feature_store()\n",
    "\n",
    "print(f\"Loaded features for {len(feature_dict)} symbols\")\n",
    "print(\"Symbols:\", list(feature_dict.keys())[:10])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 2: Extract Feature Dimensions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Get feature columns from first symbol\n",
    "sample_df = next(iter(feature_dict.values()))\n",
    "exclude_cols = ['open', 'high', 'low', 'close', 'volume', 'adj_close',\n",
    "                'symbol', 'source', 'fetch_timestamp',\n",
    "                'target', 'target_return', 'target_direction', 'target_binary']\n",
    "feature_cols = [c for c in sample_df.columns if c not in exclude_cols]\n",
    "n_features = len(feature_cols)\n",
    "\n",
    "print(f\"Number of features: {n_features}\")\n",
    "print(f\"Feature examples: {feature_cols[:10]}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 3: Initialize LinUCB Agent"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create agent\n",
    "agent = LinUCBAgent(n_features=n_features, alpha=1.0)\n",
    "print(f\"LinUCB agent initialized with {agent.n_features} features\")\n",
    "print(f\"Exploration parameter (alpha): {agent.alpha}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 4: Training Loop (Simplified Demo)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Take a small subset for quick demo\n",
    "demo_symbols = list(feature_dict.keys())[:5]\n",
    "demo_feature_dict = {sym: feature_dict[sym] for sym in demo_symbols}\n",
    "\n",
    "print(f\"Training on {len(demo_feature_dict)} symbols: {demo_symbols}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Initialize trainer\n",
    "trainer = RLTrainer(\n",
    "    agent=agent,\n",
    "    feature_store=demo_feature_dict,\n",
    "    agent_type='linucb'\n",
    ")\n",
    "\n",
    "# Train for 20 rounds (quick demo)\n",
    "print(\"\\nTraining agent...\")\n",
    "stats = trainer.train_bandit(n_rounds=20, top_k=3, horizon=1)\n",
    "\n",
    "print(f\"\\nTraining complete!\")\n",
    "print(f\"Average reward: {stats['avg_reward']:.4f}\")\n",
    "print(f\"Cumulative reward: {stats['cumulative_reward']:.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 5: Visualize Training Progress"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Plot rewards over training rounds\n",
    "plt.figure(figsize=(12, 5))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(stats['rewards_per_round'])\n",
    "plt.title('Reward per Round')\n",
    "plt.xlabel('Round')\n",
    "plt.ylabel('Reward')\n",
    "plt.grid(True)\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "cumulative = np.cumsum(stats['rewards_per_round'])\n",
    "plt.plot(cumulative)\n",
    "plt.title('Cumulative Reward')\n",
    "plt.xlabel('Round')\n",
    "plt.ylabel('Cumulative Reward')\n",
    "plt.grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 6: Test Ranking"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Prepare contexts for ranking\n",
    "contexts = {}\n",
    "for symbol, df in demo_feature_dict.items():\n",
    "    if not df.empty:\n",
    "        context = df[feature_cols].iloc[-1].values\n",
    "        context = np.nan_to_num(context, nan=0.0, posinf=0.0, neginf=0.0)\n",
    "        contexts[symbol] = context\n",
    "\n",
    "# Rank symbols\n",
    "rankings = agent.rank_symbols(contexts, top_k=None)\n",
    "\n",
    "print(\"\\nSymbol Rankings:\")\n",
    "print(\"=\"*50)\n",
    "for i, (symbol, score, confidence) in enumerate(rankings, 1):\n",
    "    print(f\"{i}. {symbol:10s} - Score: {score:8.4f} - Confidence: {confidence:.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 7: Understanding the Agent\n",
    "\n",
    "### LinUCB Algorithm\n",
    "LinUCB (Linear Upper Confidence Bound) is a contextual bandit algorithm that:\n",
    "\n",
    "1. **Maintains a linear model** for each symbol (arm)\n",
    "2. **Balances exploration vs exploitation** using UCB formula:\n",
    "   ```\n",
    "   score = θᵀx + α√(xᵀA⁻¹x)\n",
    "   ```\n",
    "   - First term: expected reward (exploitation)\n",
    "   - Second term: uncertainty bonus (exploration)\n",
    "\n",
    "3. **Updates parameters** after observing rewards:\n",
    "   - `A = A + xxᵀ` (feature covariance)\n",
    "   - `b = b + rx` (feature-reward product)\n",
    "\n",
    "### Reward Design\n",
    "In this demo, reward = future return × 100\n",
    "- Positive returns → positive rewards\n",
    "- Negative returns → negative rewards\n",
    "- Agent learns to rank symbols with higher expected returns\n",
    "\n",
    "### Why LinUCB for Ranking?\n",
    "- Fast computation (matrix operations)\n",
    "- Provides confidence estimates\n",
    "- Works well with high-dimensional features\n",
    "- No need for neural networks\n",
    "- Perfect for edge deployment"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 8: Evaluation Metrics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Evaluate agent\n",
    "eval_metrics = trainer.evaluate(top_k=3)\n",
    "\n",
    "print(\"\\nEvaluation Metrics:\")\n",
    "print(\"=\"*50)\n",
    "print(f\"Mean Return: {eval_metrics['mean_return']:.4f}\")\n",
    "print(f\"Sharpe Proxy: {eval_metrics['sharpe_proxy']:.4f}\")\n",
    "print(f\"Win Rate: {eval_metrics['win_rate']:.2%}\")\n",
    "print(f\"\\nTop Symbols: {eval_metrics['top_symbols']}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "This notebook demonstrated:\n",
    "1. Loading feature store\n",
    "2. Initializing LinUCB agent\n",
    "3. Training on small subset (20 rounds)\n",
    "4. Visualizing training progress\n",
    "5. Ranking symbols by UCB scores\n",
    "6. Evaluating agent performance\n",
    "\n",
    "### Key Takeaways:\n",
    "- LinUCB balances trying new symbols (exploration) vs selecting known good ones (exploitation)\n",
    "- The agent learns from rewards (future returns)\n",
    "- Confidence scores help us understand prediction uncertainty\n",
    "- This lightweight approach works well for edge deployment\n",
    "\n",
    "### Next Steps:\n",
    "- Train on full dataset with more rounds\n",
    "- Try Thompson Sampling or DQN agents\n",
    "- Tune hyperparameters (alpha for exploration)\n",
    "- Add portfolio-level evaluation"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}